In [ ]:
import glob
import numpy as np
import pandas as pd

# Transforming Data

In [ ]:
#Import Data
#Please insert the directory of the csv files

path = r' ' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df) 

In [ ]:
# Merge 3 data subsets into one dataframe and drop duplicate respondents

frame = pd.concat(li, keys = ['Wave1','Wave2', 'Wave3'],sort=True).drop_duplicates(subset='meta.uuid')

In [ ]:
#Assign categorical values for gender: female=0 , male=1

frame['dem.gender'].to_frame
frame['female'] = pd.get_dummies(frame['dem.gender'], drop_first=True)

In [ ]:
# 1.Define three age categories: young adults (younger than 35 years old), middle-aged adults (aged between 35 and 55 years), older adults (older than 55 years of age)
# 2.Assign age category to respondents  

age_group = []

for x in frame['dem.age'].values:
    if x < 35:
        age_group.append(1)
    elif 35 <= x and x < 55:
        age_group.append(2)
    else:
        age_group.append(3)
        
frame['age_group'] = age_group
        

In [ ]:
#Assign categorical values to level of education completed: no=0, low=1, medium=3,high=4
education_level = []

frame['dem.education_level'].to_frame

for level in frame['dem.education_level']:
    if level == 'no':
        education_level.append(0)
    elif level == 'low':
        education_level.append(1)
    elif level == 'medium':
        education_level.append(2)
    else:
        education_level.append(3)
        
frame['education_level']=education_level


In [ ]:
# Two questions to split into individual binary columns
columns=['question.DAL_positive_countries_MX_BR_VE','question.DAL_negative_countries_MX_BR_VE']

    

In [ ]:
# Create a lit of waves(from data subsets)
# Define a function that returns a dataframe per wave

waves=frame['level_0'].unique()  

def wave(wave_name):
    return frame[frame['level_0']== wave_name]
    


In [ ]:
# Iterate over the waves, columns(two questions) and each respondent (each row)
# If there is response given, split the countries by '|'
# Create a new column per question and per country in response, if not existing already, and assign default value of 0
# If a country name is in the given response, assign value of 1 to the corresponding country column

for wave_name in waves:
    wave_frame=wave(wave_name)
    for column in columns:
        for x in wave_frame[column].keys(): # iterate over keys because of dropped duplicate respondent indeces
            exists = not pd.isnull(wave_frame[column][x]) 
            if exists:                         
                countries = wave_frame[column][x].split('|')
                for country in countries:
                    col_name = column +'_'+ country.strip()
                    if (col_name) not in frame:
                        frame[col_name] = 0
                    frame.loc[(frame.index == x) & (frame['level_0'] == wave_name), col_name] = 1     
            else:
                
                wave_frame[column][x]== 0
                       

In [ ]:
# Write the new dataset to a seperate csv file 
frame.to_csv('Waves_merged.csv', index=False)

# EDA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Respondent Demographics
# Gender: 54% female
# Age: More than 50% are in the young category. The youngest respondent is 14 and the oldest respondenst is 65 years old.
# Education: Respondont group consists more of people with low to medium level education levels compared to people with no or high education levels.
frame[['dem.age','female','education_level','question.DAL_positive_countries_MX_BR_VE_China', 'question.DAL_positive_countries_MX_BR_VE_United States']].describe()

In [ ]:
# Bar graphs showing male and female repondents's answer for if US and Chinese companies have an overall positive impact in resondents' own countries
# Responses are counted per country and response options (Y/N) across three waves

f, axes = plt.subplots(3,2, figsize=(10,8), sharey=True, sharex=True)

ax1=sns.countplot( x= 'question.DAL_positive_countries_MX_BR_VE_United States', data=frame[frame['level_0']== 'Wave1'], ax=axes[0, 0], orient='v',hue='female',palette='viridis')
ax1.set_ylabel('Wave1')
ax1.set_xlabel('')
ax1.legend().remove()

ax2=sns.countplot( x='question.DAL_positive_countries_MX_BR_VE_China', data=frame[frame['level_0']== 'Wave1'],   orient='v', ax=axes[0,1], hue='female', palette='viridis')
ax2.legend(labels=[ 'male', 'female'],loc='upper right', bbox_to_anchor=(1.3,1), prop={'size': 10})
ax2.set_xlabel('')
ax2.set_ylabel('')

ax3=sns.countplot( x= 'question.DAL_positive_countries_MX_BR_VE_United States', data=frame[frame['level_0']== 'Wave2'], ax=axes[1, 0], orient='v',hue='female',palette='viridis')
ax3.set_ylabel('Wave2')
ax3.set_xlabel('')
ax3.legend().remove()

ax4=sns.countplot( x='question.DAL_positive_countries_MX_BR_VE_China', data=frame[frame['level_0']== 'Wave2'],   orient='v', ax=axes[1,1], hue='female', palette='viridis')
ax4.legend().remove()
ax4.set_xlabel('')
ax4.set_ylabel('')

ax5=sns.countplot( x= 'question.DAL_positive_countries_MX_BR_VE_United States', data=frame[frame['level_0']== 'Wave3'], ax=axes[2, 0], orient='v',hue='female',palette='viridis')
ax5.set_ylabel('Wave3')
ax5.set_xlabel('United States - Positive')
ax5.legend().remove()

ax6=sns.countplot( x='question.DAL_positive_countries_MX_BR_VE_China', data=frame[frame['level_0']== 'Wave3'],   orient='v', ax=axes[2,1], hue='female', palette='viridis')
ax6.legend().remove()
ax6.set_xlabel('China- Positive')
ax6.set_ylabel('')

plt.tight_layout()  

In [ ]:
# Bar graphs showing answers of respondents from different age groups for if US and Chinese companies have an overall positive impact in resondents' own countries
# Responses are counted per country and response options (Y/N) across three waves

f, axes = plt.subplots(3,2, figsize=(10,8), sharey=True, sharex=True)

ax1=sns.countplot( x= 'question.DAL_positive_countries_MX_BR_VE_United States', data=frame[frame['level_0']== 'Wave1'], ax=axes[0, 0], orient='v',hue='age_group',palette='viridis')
ax1.set_ylabel('Wave1')
ax1.set_xlabel('')
ax1.legend().remove()

ax2=sns.countplot( x='question.DAL_positive_countries_MX_BR_VE_China', data=frame[frame['level_0']== 'Wave1'],   orient='v', ax=axes[0,1], hue='age_group', palette='viridis')
ax2.legend(labels=['Younger', 'Middle-Aged', 'Older'], loc='upper right', bbox_to_anchor=(1.5,1), prop={'size': 12})
ax2.set_xlabel('')
ax2.set_ylabel('')

ax3=sns.countplot( x= 'question.DAL_positive_countries_MX_BR_VE_United States', data=frame[frame['level_0']== 'Wave2'], ax=axes[1, 0], orient='v',hue='age_group',palette='viridis')
ax3.set_ylabel('Wave2')
ax3.set_xlabel('')
ax3.legend().remove()

ax4=sns.countplot( x='question.DAL_positive_countries_MX_BR_VE_China', data=frame[frame['level_0']== 'Wave2'],   orient='v', ax=axes[1,1], hue='age_group', palette='viridis')
ax4.legend().remove()
ax4.set_xlabel('')
ax4.set_ylabel('')

ax5=sns.countplot( x= 'question.DAL_positive_countries_MX_BR_VE_United States', data=frame[frame['level_0']== 'Wave3'], ax=axes[2, 0], orient='v',hue='age_group',palette='viridis')
ax5.set_ylabel('Wave3')
ax5.set_xlabel('United States - Positive')
ax5.legend().remove()

ax6=sns.countplot( x='question.DAL_positive_countries_MX_BR_VE_China', data=frame[frame['level_0']== 'Wave3'],   orient='v', ax=axes[2,1], hue='age_group', palette='viridis')
ax6.legend().remove()
ax6.set_xlabel('China- Positive')
ax6.set_ylabel('')

plt.tight_layout() 
 

In [ ]:
# Bar graphs showing answers of respondents with different education levels for if US and Chinese companies have an overall positive impact in resondents' own countries
# Responses are counted per country and response options (Y/N) across three waves

f, axes = plt.subplots(3,2, figsize=(12,8), sharey=True, sharex=True)

ax1=sns.countplot( x= 'question.DAL_positive_countries_MX_BR_VE_United States', data=frame[frame['level_0']== 'Wave1'], ax=axes[0, 0], orient='v',hue='education_level',palette='viridis')
ax1.set_ylabel('Wave1')
ax1.set_xlabel('')
ax1.legend().remove()

ax2=sns.countplot( x='question.DAL_positive_countries_MX_BR_VE_China', data=frame[frame['level_0']== 'Wave1'],   orient='v', ax=axes[0,1], hue='education_level', palette='viridis')
ax2.legend(labels=['No Education', 'Low-level Education', 'Medium-level Education', 'High-level Education'], loc='upper right', bbox_to_anchor=(1.6,1), prop={'size': 10})
ax2.set_xlabel('')
ax2.set_ylabel('')

ax3=sns.countplot( x= 'question.DAL_positive_countries_MX_BR_VE_United States', data=frame[frame['level_0']== 'Wave2'], ax=axes[1, 0], orient='v',hue='education_level',palette='viridis')
ax3.set_ylabel('Wave2')
ax3.set_xlabel('')
ax3.legend().remove()

ax4=sns.countplot( x='question.DAL_positive_countries_MX_BR_VE_China', data=frame[frame['level_0']== 'Wave2'],   orient='v', ax=axes[1,1], hue='education_level', palette='viridis')
ax4.legend().remove()
ax4.set_xlabel('')
ax4.set_ylabel('')

ax5=sns.countplot( x= 'question.DAL_positive_countries_MX_BR_VE_United States', data=frame[frame['level_0']== 'Wave3'], ax=axes[2, 0], orient='v',hue='education_level',palette='viridis')
ax5.set_ylabel('Wave3')
ax5.set_xlabel('United States - Positive')
ax5.legend().remove()

ax6=sns.countplot( x='question.DAL_positive_countries_MX_BR_VE_China', data=frame[frame['level_0']== 'Wave3'],   orient='v', ax=axes[2,1], hue='education_level', palette='viridis')
ax6.legend().remove()
ax6.set_xlabel('China- Positive')
ax6.set_ylabel('')

plt.tight_layout() 
 

In [ ]:
# Table of the results (in percentages) for the positive perception of China and the United States, segmented by wave, country, gender, education level and three age groups
# Each cell shows the percentage of positive responses per condition that the respondents are in.
# For exmple, first cell shows that 23% of respondents who are young females with high education levels and who completed the survey in the first wave in Brazil have positive perception for the Chinese companies.

table = pd.pivot_table(frame, values=['question.DAL_positive_countries_MX_BR_VE_United States','question.DAL_positive_countries_MX_BR_VE_China'], index=['level_0','dem.gender', 'dem.education_level', 'age_group'],columns=['dem.country_code'], fill_value=0,aggfunc='count').div(len(frame.index)).mul(100).round(2)
table

# Data Analysis

In [ ]:
from scipy import stats

In [ ]:
# Correlations between gender, age group and education level of the respondents, and their perception of the US and China across three waves
# Although some of the correlations are significant p<0.01, effect sizes are too small approximating to 0.
# Results show taht data collection in different waves does not have a significant impact on respondents' perception of the US and China.

corr_table=[]


for dv in ['question.DAL_positive_countries_MX_BR_VE_United States', 'question.DAL_positive_countries_MX_BR_VE_China']:
    for iv in ['female', 'age_group', 'education_level']:
        for wave_name in waves:
            wave_frame=wave(wave_name)
            
            corr_table.append([stats.pearsonr(wave_frame[iv],wave_frame[dv])])
            print(wave_name,iv,dv + str(stats.pearsonr(wave_frame[iv],wave_frame[dv])))

corr_table=np.reshape(corr_table, (18,2)) 

In [ ]:
# Pairwise correlations for the columns with demographics data aggregated over the waves
# There is a significant positive correlation between having positive perception of the US companies and having positive perception of Chineese companies, r=0.39, p=0.0.
# This indicates that as the positive perception for the US companies increases, positive perception of Chinese companies increases as well.
# Also, as the repondents think that the US companies have more negative impact on their countries, then they are more likely to think that Chinese companies have negative impact on their countries as well.

log_reg_data=frame[['level_0','female', 'age_group','education_level','question.DAL_positive_countries_MX_BR_VE_United States','question.DAL_positive_countries_MX_BR_VE_China']]
log_reg_data.corr()

In [ ]:
stats.pearsonr(log_reg_data['question.DAL_positive_countries_MX_BR_VE_United States'], log_reg_data['question.DAL_positive_countries_MX_BR_VE_China'])

In [ ]:
# Building Logistic Regression Model for the Perception of US Companies


In [ ]:
waves=[]
for i in log_reg_data['level_0'].values:
    
    if i == 'Wave1':
        waves.append(1)
    elif i == 'Wave2':
        waves.append(2)
    else:
        waves.append(3)  
log_reg_data['level_0']=waves

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split the data into test and train data
X=log_reg_data.drop(['question.DAL_positive_countries_MX_BR_VE_United States','question.DAL_positive_countries_MX_BR_VE_China'], axis=1)
y= log_reg_data['question.DAL_positive_countries_MX_BR_VE_United States']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
#Model Creation
log_model=LogisticRegression()

In [ ]:
# Fit the test data to the model
log_model.fit(X_train,y_train)

In [ ]:
# Predict responses based on the model using test data
predictions=log_model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

In [ ]:
# Building Logistic Regression Model for the Perception of Chineese Companies

In [ ]:
# Split the data into test and train data
y=frame['question.DAL_positive_countries_MX_BR_VE_China']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
#Model Creation
log_model_CH=LogisticRegression()

In [ ]:
# Fit the test data to the model
log_model_CH.fit(X_train,y_train)

In [ ]:
# Predict responses based on the model using test data
pred=log_model_CH.predict(X_test)

In [ ]:
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))

In [ ]:
# Both of the models perfomed poorly with precison values equal or less than 0.60.
# Negative answers (0) could not be explained by the model.